<a href="https://colab.research.google.com/github/Hepheir/Paper-Scanning/blob/AlexNet/MachineLearning/AlexNet/CIFAR-10%20using%20AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10

In [1]:
#@title Imports

import numpy as np

from tensorflow.keras import backend
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

---

## Local Response Normalization

Keras에는 Local Response Normalization 명령이 없어졌다고 한다. [이 곳](https://datascienceschool.net/view-notebook/d19e803640094f76b93f11b850b920a4/)에 구현된 LRN명령을 가져와 사용한다.

참고:
* [Difference between Local Response Normalization and Batch Normalization](https://towardsdatascience.com/difference-between-local-response-normalization-and-batch-normalization-272308c034ac)
* [AlexNet - Data Science School](https://datascienceschool.net/view-notebook/d19e803640094f76b93f11b850b920a4/)

In [2]:
#@title Local Response Normalization

class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = backend.square(x)
        pooled = backend.pool2d(squared, (self.n, self.n), strides=(1,1), padding="same", pool_mode='avg')
        summed = backend.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * backend.repeat_elements(summed, f, axis=3)
        denom = backend.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape 

---

## AlexNet using Keras


In [3]:
#@title Build Model

model = Sequential([
    # The first convolutional layer
    Conv2D(
        input_shape=(32, 32, 3),
        kernel_size=(5,5),
        filters=96,
        padding='valid',
        activation='relu'),
    LocalResponseNormalization(),
    MaxPooling2D(
        pool_size=(3,3),
        strides=(2,2),
        padding='valid'),
    # The second convolutional layer
    Conv2D(
        filters=256,
        kernel_size=(3,3),
        padding='valid',
        activation='relu'),
    LocalResponseNormalization(),
    # The third convolutional layer
    Conv2D(
        filters=384,
        kernel_size=(3,3),
        padding='valid',
        activation='relu'),
    # The fourth convolutional layer
    Conv2D(
        filters=384,
        kernel_size=(3,3),
        padding='valid',
        activation='relu'),
    # Connect between Convolutional layers and Fully-Connected layers
    Flatten(),
    # 6th Layer: Fully-Connected
    Dense(units=256, activation='relu'),
    Dropout(rate=0.4),
    # 7th Layer: Fully-Connected
    Dense(units=128, activation='relu'),
    Dropout(rate=0.4),
    # 8th Layer: Fully-Connected
    Dense(units=10, activation='relu'),
    # Output Layer
    Dense(units=10, activation='softmax')
])

model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 96)        7296      
_________________________________________________________________
local_response_normalization (None, 28, 28, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 256)       221440    
_________________________________________________________________
local_response_normalization (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 384)         885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 384)         1

In [4]:
#@title Download CIFAR-10 dataset

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [ ]:
import cv2



In [ ]:

# 데이터셋 전처리 : 데이터 정규화 및 one-hot 인코딩
x_train = np.reshape(x_train / 255.0, tuple( [x_train.shape[0]] + list(model.input_shape[1:])))
y_train = to_categorical(y_train)
x_test = np.reshape(x_test / 255.0, tuple([x_test.shape[0]] + list(model.input_shape[1:])))
y_test = to_categorical(y_test)

In [6]:
#@title Train Model

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = 'CIFAR-10/training/cp-{epoch:04d}.ckpt'
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    verbose=1,
    save_weights_only=False
)

model.fit(x_train, y_train,
    batch_size=100,
    epochs=3,
    callbacks=[
        checkpoint_callback
    ])
model.evaluate(x_test, y_test)
model.save('CIFAR-10/model/cifar-10.alexnet')

Epoch 1/3
499/500 [============================>.] - ETA: 0s - loss: 2.3035 - accuracy: 0.1000
Epoch 00001: saving model to CIFAR-10/training/cp-0001.ckpt
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: CIFAR-10/training/cp-0001.ckpt\assets
500/500 [==============================] - 13s 25ms/step - loss: 2.3035 - accuracy: 0.1000
Epoch 2/3
500/500 [==============================] - ETA: 0s - loss: 2.3032 - accuracy: 0.1000
Epoch 00002: saving model to CIFAR-10/training/cp-0002.ckpt
INFO:tensorflow:Assets written to: CIFAR-10/training/cp-0002.ckpt\assets
500/500 [==============================] - 13s 25ms/step - loss: 2.3032 - accuracy: 0.1000
Epoch 3/3
498/500 [============================>.] - ETA: 0s - loss: 2.3030 - accuracy: 0.1000
Epoch 00003: saving model to CIFAR-10/training/cp-0003.ckpt
INFO:tensorflow:Assets written to: CIFAR-10/training/cp-0003.ckpt\assets
313/313 [==============================] - 1s 5ms/ste